<a href="https://colab.research.google.com/github/BakuDoriya-KatsZuku/Whatsapp_Chat_Reports/blob/main/WhatsApp_Chat_Reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import datetime
from textblob import TextBlob

with open("Path of .txt file containing whatsApp chats") as f:
    lines = f.readlines()

dates, times, names, messages = [], [], [], []

for line in lines:
    line.strip("\n")
    if " - " in line:
        parts = line.split(" - ")
        if len(parts) == 2:
            date_time = parts[0].strip().split(', ')
            if len(date_time) == 2:
                date, time = date_time
                x = parts[1].split(":")
                name = ''
                if len(x) == 2:
                  name, message = x
                  dates.append(date)
                  times.append(time)
                  names.append(name)
                  messages.append(message)

df = pd.DataFrame({
    "Date": dates,
    "Time": times,
    "Name": names,
    "Message": messages
})

df["Message"] = df["Message"].str.strip("\n")

In [ ]:
x = dict(df["Name"].value_counts())
x = pd.DataFrame(x.items(), columns = ["Name", "Count"])
fig = px.bar(x, x = "Name", y = "Count", title = "Number of Messages")
fig.show()

In [ ]:
#Make DF words and count (Keeping this so that we can also use for plotting)
x = {}
for i in range(df.shape[0]):
  words = df["Message"].iloc[i].split()
  for word in words:
    if word not in x:
      x[word] = 1
    else:
      x[word] += 1
x = pd.DataFrame(x.items(), columns = ["Word", "Count"])

#Removing Media
z = x[(x["Word"] == "<Media") | (x["Word"] == "omitted>")].index
x.drop(z, inplace = True)
x = x.sort_values(by = 'Count', ascending = False)
print("Most Messaged Words")
print(x.to_string(index = False))

In [ ]:
#Making Dictionary for words with lowercase (IDK why groupby does not give result. It just prints unkown characters)
x = {}
for i in range(df.shape[0]):
  words = df["Message"].iloc[i].split()
  for word in words:
    word = word.lower()
    if word not in x:
      x[word] = 1
    else:
      x[word] += 1
x = pd.DataFrame(x.items(), columns = ["Word", "Count"])
x = x.sort_values(by = 'Count', ascending = False)

print(x.loc[x["Word"].isin(['bsdk','fuck', 'mc', 'bc', 'shekhu', 'yugraj', "f", "Manas"])])

In [ ]:
new_df = df[df.Message != "<Media omitted>"] #Removing Medai ommitted so that they do not hamper word counts

z = df["Name"].unique() #List of unique names

#Dictionary to count the number of words in each name
x = {}
for name in z:
  x[name] = 0
for i in range(new_df.shape[0]):
  x[df["Name"].iloc[i]] += len(new_df["Message"].iloc[i].split())  

#Sortig the Dictionary in reverse order
keys = list(x.keys())
values = list(x.values())
sorted_value_index = np.argsort(values)
x = {keys[i]: values[i] for i in sorted_value_index}
x = dict(reversed(list(x.items())))

#Plot
x = pd.DataFrame(x.items(), columns = ["Name", "Count"])
fig = px.bar(x, x = "Name", y = "Count", title = "Number of Words")
fig.show()

In [2]:
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y', errors='coerce') #Forcing all the values in the column to become in that format else NAN
if df['Date'].isnull().sum() > 0:
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
df["Date"] = df["Date"].dropna() #Drop all useless values
df['Month_Year'] = df['Date'].dt.strftime('%B %Y') #Converting to Month year for better looking on the graph
grouped = df.groupby(['Name', 'Month_Year', 'Date']).count() #Grouping these values ie making them the primary key

# Performing Operations for each and every person
for name in grouped.index.levels[0]:
    sub_df = grouped.loc[name].reset_index()
    sub_df = sub_df.sort_values(by='Date')
    #Replacing the missing date values ie where the person did not message and putting the message count to 0 over there
    sub_df = sub_df.set_index("Date")
    sub_df = sub_df.resample("D").asfreq() #Replace missing date values (Can change only if index is the column whose dates you want to insert ie all the values there have to be unique)
    sub_df["Message"].fillna(0, inplace=True)
    sub_df = sub_df.reset_index() #Change index again
    #Plot
    fig = px.line(sub_df, x = 'Date', y = 'Message', title = name)
    fig.show()

#Same but not bringing back in the dates (Can't do because they repeat because of repeating name. One way is the create different dataframes and then concat them and then plot with color = "name" but takes too much time so did not do it)
grouped = grouped.reset_index()
grouped = grouped.sort_values(by='Date')
fig = px.line(grouped, x = 'Date', y = 'Message', color = 'Name', title = 'Everyone')
fig.show()

#Same thing but for the whole group so ignoring the names
grouped = df.groupby(['Date']).count()
grouped = grouped.reset_index()
grouped = grouped.sort_values(by = 'Date')
grouped = grouped.set_index("Date")
grouped = grouped.resample("D").asfreq()
grouped["Message"].fillna(0, inplace=True)
grouped = grouped.reset_index()
fig = px.line(grouped, x = 'Date', y = 'Message', title = 'Overall messages of the group')
fig.show()

In [ ]:
#Using textblob for sentimental analysis (IDK how to use it so chatGPT(Orz))
df['Sentiment'] = df['Message'].apply(lambda x: TextBlob(x).sentiment.polarity)
grouped = df.groupby(['Name'])
sentiment_scores = grouped['Sentiment'].mean() #Finding mean because just 1 number of each person look good

#Plot
fig = px.bar(sentiment_scores, x=sentiment_scores.index, y=sentiment_scores, title='Sentiment Scores by Person')
fig.show()

In [ ]:
#Instead of messages did the same process for number of words
df['Word Count'] = df['Message'].apply(lambda x: len(x.split()))
grouped = df.groupby(['Date', 'Name'])
word_count_per_day = grouped['Word Count'].sum().reset_index()

#IDK what this is chatGPT(Orz)
for name, sub_df in word_count_per_day.groupby('Name'):
    fig = px.line(sub_df, x='Date', y='Word Count', title=name)
    fig.show()

overall_word_count = word_count_per_day.groupby('Date').sum().reset_index()

fig = px.line(overall_word_count, x='Date', y='Word Count', title='Overall Word Count of the group')
fig.show()


In [ ]:
#Instead of Date using hours as a metric to see which time of the day is most active
def convert(string):
      if string[-2:] == "AM" and string[:2] == "12":
         return "00" + string[2:-2]
      elif string[-2:] == "AM":
         return string[:-2]
      elif string[-2:] == "PM" and string[:2] == "12":
         return string[:-2]
      else:
          return str(int(string[:2]) + 12) + string[2:8]

def extract_hour(time_str):
    #Returns at which hour the message was sent. Rounds off the nearest hour. Could be ciel as well as floor value
    if time_str[-1:] >= "30":
      if time_str[1] == "9":
        x = list(time_str)
        x[1] = "0"
        time_str = ''.join(x)
        if time_str[0] == "1":
          x = list(time_str)
          x[0] = "2"
          time_str = ''.join(x)
        else:
          x = list(time_str)
          x[0] = "1"
          time_str = ''.join(x)
      elif time_str[0:2] == '23':
          x = list(time_str)
          x[0] = "0"
          x[1] = "0"
          time_str = ''.join(x)
    return time_str[0:2]

df['Hour'] = df['Time'].apply(extract_hour) #Making new column of hours
grouped = df.groupby(["Name", "Hour"]).count()

#Individual
for name in grouped.index.levels[0]:
  sub_df = grouped.loc[name].reset_index()
  sub_df = sub_df.sort_values(by = "Hour")
  fig = px.line(sub_df, x = 'Hour', y = 'Message', title = name)
  fig.show()

#Whole Group
grouped = df.groupby(["Hour"]).count()
grouped = grouped.sort_values(by = "Hour")
fig = px.line(sub_df, x = 'Hour', y = 'Message', title = "All Messages of the group")
fig.show()

In [ ]:
def awc(x, y):
  for name in list(x.keys()):
    x[name] = (x[name] / y[name])
  return x

y = dict(df["Name"].value_counts())

new_df = df[df.Message != "<Media omitted>"] #Removing Medai ommitted so that they do not hamper word counts

z = df["Name"].unique() #List of unique names

#Dictionary to count the number of words in each name
x = {}
for name in z:
  x[name] = 0
for i in range(new_df.shape[0]):
  x[df["Name"].iloc[i]] += len(new_df["Message"].iloc[i].split())  

out = awc(x, y)

keys = list(x.keys())
values = list(x.values())
sorted_value_index = np.argsort(values)
x = {keys[i]: values[i] for i in sorted_value_index}
x = dict(reversed(list(x.items())))

print("Average Words per Message\n")
keys = list(x.keys())
for name in keys:
  print(name + ": " + str(x[name]) + "\n")